In [ ]:
# Imports
from confluent_kafka import Consumer
from time import sleep
import uuid
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException
import json
from confluent_kafka.admin import AdminClient, NewTopic
from pyspark.sql.functions import when
from pyspark.sql.functions import col
import random as r

# Get config
from config import user
from config import password

In [ ]:
# Mount point through Oauth security.
storageAccount = "gen10datafund2205"
storageContainer = "group5container"
clientSecret = "-ZS8Q~NwOKfwEpVOg3Teb1pPtxDbz616XjlXLbuU"
clientid = "2ca50102-5717-4373-b796-39d06568588d"
mount_point = "/mnt/group5/cgm" 

configs = {"fs.azure.account.auth.type": "OAuth",
       "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
       "fs.azure.account.oauth2.client.id": clientid,
       "fs.azure.account.oauth2.client.secret": clientSecret,
       "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
       "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

# Unmount if exists
try: 
    dbutils.fs.unmount(mount_point)
except:
    pass

# Mount to database
dbutils.fs.mount(
    source = "abfss://"+storageContainer+"@"+storageAccount+".dfs.core.windows.net/",
    mount_point = mount_point,
    extra_configs = configs)

In [ ]:
# Get data
df = spark.read.options(
    inferSchema='True',
    delimiter=',',
    header='True'
    ).csv('/mnt/group5/cgm/CGM_Data.csv')

In [ ]:
patient1 = df.filter(df.PtID == 1).sort('DeviceDtTm')
patient1 = patient1.filter(patient1['DeviceDtTm'] >= '2000-06-09 00:00:00')

patient2 = df.filter(df.PtID == 14).sort('DeviceDtTm')
patient2 = patient2.filter(patient2['DeviceDtTm'] >= '2000-03-26 00:00:00')

patient3 = df.filter(df.PtID == 33).sort('DeviceDtTm')
patient3 = patient3.filter(patient3['DeviceDtTm'] >= '2000-05-14 00:00:00')

In [ ]:
#KAFKA variables, Move to the OS variables or configuration
confluentClusterName = "stage3talent"
confluentBootstrapServers = "pkc-ldvmy.centralus.azure.confluent.cloud:9092"
confluentTopicName = "group5cgm"
schemaRegistryUrl = "https://psrc-gq7pv.westus2.azure.confluent.cloud"
confluentApiKey = "YHMHG7E54LJA55XZ"
confluentSecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"
confluentRegistryApiKey = "YHMHG7E54LJA55XZ"
confluentRegistrySecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"

In [ ]:
# Get error message functions
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)


def acked(err, msg):
    """ 
        Error callback is used for generic issues for producer errors. 
        
        Parameters:
            err (err): Error flag.
            msg (str): Error message that was part of the callback.
    """
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))

In [ ]:
# Create producer

#Kakfa Class Setup.
p = Producer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(1),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest',
    'error_cb': error_cb,
})

In [ ]:
# Stream data
keep_going = True
j = 0
for i in range(288):
    
    # Create dictionary object with that data row
    datum1 = dict()
    datum2 = dict()
    datum3 = dict()
    datum = []
    
    for header in patient1.columns:
        datum1[header] = patient1.collect()[j][header]
    datum1['DeviceDtTm'] = datum1['DeviceDtTm'].strftime('%m/%d/%Y, %H:%M:%S')
    
    for header in patient2.columns:
        datum2[header] = patient2.collect()[j][header]
    datum2['DeviceDtTm'] = datum2['DeviceDtTm'].strftime('%m/%d/%Y, %H:%M:%S')
    
    for header in patient3.columns:
        datum3[header] = patient3.collect()[j][header]
    datum3['DeviceDtTm'] = datum3['DeviceDtTm'].strftime('%m/%d/%Y, %H:%M:%S')
    
    datum.append(datum1)
    datum.append(datum2)
    datum.append(datum3)
    
    # Produce and flush data
    p.produce(confluentTopicName, json.dumps(datum))
    p.flush()
    
    # Print flush info
    print(f'Current step: {j}\n({datum})\n')
    j += 1

In [ ]:
patient4 = df.filter(df.PtID == 1).sort('DeviceDtTm')
patient4 = patient4.filter(patient4['DeviceDtTm'] >= '2000-06-10 00:00:00')

patient5 = df.filter(df.PtID == 14).sort('DeviceDtTm')
patient5 = patient5.filter(patient5['DeviceDtTm'] >= '2000-03-27 00:00:00')

patient6 = df.filter(df.PtID == 33).sort('DeviceDtTm')
patient6 = patient6.filter(patient6['DeviceDtTm'] >= '2000-05-15 00:00:00')

In [ ]:
# Stream data
keep_going = True
j = 0
for i in range(288):
    
    # Create dictionary object with that data row
    datum4 = dict()
    datum5 = dict()
    datum6 = dict()
    datum = []
    
    for header in patient4.columns:
        datum4[header] = patient4.collect()[j][header]
    datum4['DeviceDtTm'] = datum4['DeviceDtTm'].strftime('%m/%d/%Y, %H:%M:%S')
    
    for header in patient5.columns:
        datum5[header] = patient5.collect()[j][header]
    datum5['DeviceDtTm'] = datum5['DeviceDtTm'].strftime('%m/%d/%Y, %H:%M:%S')
    
    for header in patient6.columns:
        datum6[header] = patient6.collect()[j][header]
    datum6['DeviceDtTm'] = datum6['DeviceDtTm'].strftime('%m/%d/%Y, %H:%M:%S')
    
    datum.append(datum4)
    datum.append(datum5)
    datum.append(datum6)
    
    # Produce and flush data
    p.produce(confluentTopicName, json.dumps(datum))
    p.flush()
    
    # Print flush info
    print(f'Current step: {j}\n({datum})\n')
    j += 1
    sleep(10) # Sleep for 10 seconds